In [8]:
import datetime
def main():
    service = get_calendar_service()

    calId = 'ryan.jeffery.young@gmail.com'

    # call APi
    now = datetime.datetime.utcnow().isoformat() + 'Z' # z for UTC time
    tmr = datetime.datetime.utcnow() + timedelta(days=2)
    tmr_utc = tmr.isoformat() + 'Z'

    body = {
           "timeMin": now,
           "timeMax": tmr_utc,
           "items": [{"id": calId}]
       }
    
    freebusy_result = service.freebusy().query(body=body).execute()

    freebusy = freebusy_result[u'calendars']

    busyList = freebusy[calId]['busy']


    busy_cal = {}
    for cal_name in freebusy:
      busy_cal.update(freebusy[cal_name])

    print((busyList[0])['start'])


## Authentication

In [1]:
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request


In [2]:
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

CREDENTIALS_FILE = 'credentials.json'

In [3]:
creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                CREDENTIALS_FILE, SCOPES)
                creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
service = build('calendar', 'v3', credentials=creds)

## Call API


In [4]:
import datetime
from datetime import timedelta
from datetimerange import DateTimeRange   
from setup import get_calendar_service

In [5]:
calId = 'ryan.jeffery.young@gmail.com'

In [6]:
# get todays date
today = datetime.date.today()
tomorrow = today + timedelta(days=1)

# convert date into datetime object at start of day, which is 800 UTC
start = datetime.datetime(today.year, today.month, today.day, 8, 1, 0) # 8am UTC is 12am PST
# convert to ISO format then append 'Z' for UTC time
start_UTC = start.isoformat() + 'Z'

end = datetime.datetime(today.year, today.month, tomorrow.day, 8, 0, 0) #7:59am UTC is 11:59pm PST
end_UTC = end.isoformat() + 'Z'

body = {
           "timeMin": start_UTC,
           "timeMax": end_UTC,
           "timeZone": 'PST',
           "items": [{"id": calId}]
       }
freebusy_result = service.freebusy().query(body=body).execute() # returns a dictionary 

freebusy_result

{'kind': 'calendar#freeBusy',
 'timeMin': '2020-12-08T08:01:00.000Z',
 'timeMax': '2020-12-09T08:00:00.000Z',
 'calendars': {'ryan.jeffery.young@gmail.com': {'busy': []}}}

In [15]:
busy_list = freebusy_result['calendars'][calId]['busy'] # returns a list of busy times (in UTC)
busy_list

[{'start': '2020-12-04T10:00:00-08:00', 'end': '2020-12-04T10:30:00-08:00'},
 {'start': '2020-12-04T11:00:00-08:00', 'end': '2020-12-04T12:00:00-08:00'},
 {'start': '2020-12-04T13:30:00-08:00', 'end': '2020-12-04T14:30:00-08:00'},
 {'start': '2020-12-04T15:30:00-08:00', 'end': '2020-12-04T16:00:00-08:00'}]

In [9]:
busy_times = []
busy_times.clear

<function list.clear()>

In [10]:
for event in busy_list:
    # get response from API
    start_int = event['start']
    end_int = event['end']

    # convert responses into HH:MM
    start_hr_min = start_int[11:16]
    end_hr_mind = end_int[11:16]

    # add start-end to array of busy times
    busy_times.append([start_hr_min, end_hr_mind])
busy_times

[['09:00', '09:15']]

In [142]:
for busy_timeslot in busy_times:
    # set time range as start of busy time to end of busy time, for each busy event in list
    time_range = DateTimeRange(busy_timeslot[0], busy_timeslot[1])
    
    # set current time
    now =  datetime.datetime.utcnow().isoformat()
    
    print(time_range)
    # check if current time within any set of start-end times
    if now in time_range:
        print('Busy until', busy_timeslot[1])
    else:
        print('Not busy right now!')
        print(now)


2020-11-24T12:00:00 - 2020-11-24T16:00:00
Not busy right now!
2020-11-25T04:23:29.518847
2020-11-24T19:45:00 - 2020-11-24T21:00:00
Not busy right now!
2020-11-25T04:23:29.519848
2020-11-24T22:30:00 - 2020-11-24T00:00:00


ValueError: time inversion found: 2020-11-24 22:30:00 > 2020-11-24 00:00:00

In [11]:
# set time range as start of today to end of day
time_range = DateTimeRange(start_UTC, end_UTC)
# get current time, in UTC
now =  datetime.datetime.utcnow().isoformat() + '+00:00'
future = datetime.datetime(2020, 11, 26, 8, 0, 0).isoformat() + '+00:00'

if now in time_range:
    print('yuh')
else:
    print('nah')


yuh


In [32]:
beg = '2020-12-04T18:00:00Z'
end = '2020-12-04T18:30:00Z'

time_range = DateTimeRange(beg, end)

print(now in time_range)

False


In [20]:
now = datetime.datetime.utcnow().isoformat() + 'Z'
now

'2020-12-05T02:38:57.678511Z'

In [36]:
if now > end:
    print('yuh')

yuh


In [33]:
end

'2020-12-04T18:30:00Z'